**IMPORTING LIBRARIES**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.preprocessing.image import  ImageDataGenerator
import skimage.io
import skimage.transform
from skimage.segmentation import slic,quickshift,mark_boundaries
import skimage.color as color
import copy
import sklearn.metrics


**IMPORTING INCEPTIONV3 MODEL**

In [2]:
model=tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

96116736/96112376 [==============================] - 1s 0us/step


**DISPLAYING MODEL ATTRIBUTES**

In [3]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

**IMPORTING IMAGE AND PRE-PROCESSING**

In [4]:
from keras.preprocessing import image
image_path='/content/drive/MyDrive/dog and cat.jpg'
test_image=skimage.io.imread(image_path)
test_image=skimage.transform.resize(test_image, (299,299)) 
test_image = (test_image - 0.5)*2
plt.imshow(test_image/2+0.5)


FileNotFoundError: ignored

**PREDICTING RESULTS AND EXTRACTING TOP 5 CLASSES**

In [ ]:
preds =model.predict(test_image[np.newaxis,:,:,:])
tf.keras.applications.imagenet_utils.decode_predictions(preds,top=5)[0]

In [ ]:
top= preds[0].argsort()[-5:][::-1]
top  

In [ ]:
seg_quick=quickshift(test_image,kernel_size=4,max_dist=200,ratio=0.2)
super_p = np.unique(seg_quick).shape[0]
super_p

**DISPLAYING SEGMENTS OF IMAGE**

In [ ]:
fig2,ax2=plt.subplots(1,1,sharex=True,sharey=True)
ax2.imshow(mark_boundaries(test_image/2+0.5,seg_quick))
plt.tight_layout()
plt.show()
plt.imshow(test_image/2+0.5)

**CREATING A RANDOM MASKS FOR SEGMENTED IMAGE**

In [ ]:
perturb = 150
perturbations = np.random.binomial(1, 0.5, size=(perturb,super_p))
perturbations[0]

**DEACTIVATING SUPERPIXELS**

In [ ]:
def perturb_image(img,perturbation,segments):
  active_pixels = np.where(perturbation == 1)[0]
  mask = np.zeros(segments.shape)
  for active in active_pixels:
      mask[segments == active] = 1 
  perturbed_image = copy.deepcopy(img)
  perturbed_image = perturbed_image*mask[:,:,np.newaxis]
  return perturbed_image

In [ ]:
plt.imshow(perturb_image(test_image/2+0.5,perturbations[0],seg_quick))

**MAKING PREDICTIONS ON PERTURBED IMAGE**

In [ ]:
predictions = []
for pert in perturbations:
  perturbed_img = perturb_image(test_image,pert,seg_quick)
  pred =model.predict(perturbed_img[np.newaxis,:,:,:])
  predictions.append(pred)

predictions = np.array(predictions)
predictions.shape

**COMPUTING THE DISTANCE BETWEEN ORIGINAL AND PERTURBED IMAGE**

In [ ]:
original = np.ones(super_p)[np.newaxis,:] 
distances = sklearn.metrics.pairwise_distances(perturbations,original, metric='cosine').ravel()
distances.shape

In [ ]:
kernel_width = 0.25
weights = np.sqrt(np.exp(-(distances**2)/kernel_width**2)) #Kernel function
weights.shape

**FITTING A LINEAR MODEL**

In [ ]:
from sklearn.linear_model import LinearRegression
class_to_explain = top[0]
simpler_model = LinearRegression()
simpler_model.fit(X=perturbations, y=predictions[:,:,class_to_explain], sample_weight=weights)
coeff = simpler_model.coef_[0]
coeff

**FINDING THE IMPORTANT SUPERPIXELS**

In [ ]:
num_top_features = 4
top_features = np.argsort(coeff)[-num_top_features:] 
top_features

**DISPLAYING TOP FEATURES OF IMAGE**

In [ ]:
mask = np.zeros(super_p) 
mask[top_features]= True 
plt.imshow(perturb_image(test_image/2+0.5,mask,seg_quick))
